In [111]:
%matplotlib inline
import numpy as np
import pandas as pd
import urllib2
import urllib


In [108]:
path = "/Users/sippiejason/datascience/webscraping/realestate/data/"

addrs = pd.read_csv(path+"addr.txt")


614 Appledoorn Cir
527 Cardinal Ln
132 Alpine Ridge Dr
423 Appeldoorn Cir
538 Appeldoorn Cir
283 Allens Creek Rd
617 Appeldoorn Cir
457 Allens Creek Rd
718 Appeldoorn Cir
651 Allens Creek Rd
688 Allens Creek Rd
738 Appeldoorn Cir
26 Robinson Creek Rd
15 Ravenna St
934 Allens Creek Rd
102 Marretta Ln
2 Whispering Rock Ln
69 Wanoca Ave
9 S Oak Forest Dr
304 Solitary Meadow Cir
378 School Rd E
754 Wayneview Dr
85 School Rd E #E
201 Waynewood Dr
651 School Rd
1695 Walnut Creek Rd
212 Tiger Lily Ln
363 School Rd E
60 Skyview Dr
54 Tam Dr
300 School Rd E
286 School Rd E
253 School Rd E
75 Soaring Mtn Dr
73 Wanoca Ave
5 S Oak Forest Dr
28 S Oak Forest Dr
66 Woodview Dr
140 Woodside Cir
81 Sheridan Rd
7 Sheridan Rd
116 Woods Edge Rd
74 Robinson Ave
50 Woody Ln
77 Wren Way
430 Wren Way
32 Sayles Rd
155 Treetop Ln
160 Treetop Ln
286 Wren Way
47 Sheridan Rd
21 S Oak Forest Dr
20 S Oak Forest Dr
19 S Oak Forest Dr
69 Robinson Ave
146 Zemery Ln
27 Sayles Rd
15 Woodchuck Way
87 Willow Rd #13
87 Will

In [182]:
allLocs = pd.DataFrame()
missedLocs = pd.DataFrame()


for i in addrs.index:

    qsDict = {'street': addrs.loc[i, "streetAddr"],
                 'city': addrs.loc[i, "cityName"],
                 'state': 'NC'}

    prefix = "https://geocoding.geo.census.gov/geocoder/geographies/address?"
    suffix = "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
    qs = urllib.urlencode(qsDict)

    try:
        response = urllib2.urlopen(prefix + qs + suffix)
        html = response.read()
        j = json.loads(html)
    except: 
        print "error contacting server"

    try:
        if i%50 == 0: print "Trying row " + str(i)
        blocks = j['result']['addressMatches'][0]['geographies']['Census Blocks'][0]
        blocks["addrQuery"] = addrs.loc[i, "addrQuery"]
        allLocs = allLocs.append(pd.DataFrame(blocks, index=[blocks["OBJECTID"]]))
    except:
        print "Couldn\'t get " + addrs.loc[i, "addrQuery"]
        missedLocs = missedLocs.append(addrs.iloc[i,:])
        

#blocksDF



Trying row 0
Couldn't get 614 Appledoorn Cir Asheville NC 28803
Couldn't get 423 Appeldoorn Cir Asheville NC 28803
Couldn't get 538 Appeldoorn Cir Asheville NC 28803
Couldn't get 617 Appeldoorn Cir Asheville NC 28803
Couldn't get 718 Appeldoorn Cir Asheville NC 28803
Couldn't get 738 Appeldoorn Cir Asheville NC 28803
Couldn't get 2 Whispering Rock Ln Asheville NC 28803
Couldn't get 363 School Rd E Asheville NC 28803
Couldn't get 155 Treetop Ln Asheville NC 28803
Couldn't get 160 Treetop Ln Asheville NC 28803
Trying row 50
Couldn't get 307 S Braeside Ct Asheville NC 28803
Couldn't get 303 S Braeside Ct Asheville NC 28803
Couldn't get 685 Winding Creek Dr Waynesville NC 28786
Couldn't get 145 Winding Creek Dr Waynesville NC 28786
Couldn't get 105 Winding Creek Dr Waynesville NC 28786
Couldn't get 65 Welch St Waynesville NC 28786
Couldn't get 11 Winters Edge Ln Asheville NC 28803
Couldn't get 104 Woodfield Dr Asheville NC 28803
Trying row 100
Couldn't get 52 Winterwind Dr Asheville NC 288

In [185]:
allLocs.to_csv(path+"addrGeo.txt")
missedLocs.to_csv(path+"addrGeoMissed.txt")

In [184]:
response2 = urllib2.urlopen()
metric = "B25081_008E"
blockGroup = 
stateCode = 
countyCode = 
tractCode =

prefix = "http://api.census.gov/data/2015/acs5?get=NAME,"
bg = "&for=block+group:" & blockGroup
loc = "&in=state:" + stateCode + "+county:" + countyCode + "+tract:" + tractCode 
key = "&key=49bdc2de261c9822e82e5e1c8c1b288ed173bd90"

url = prefix + metric + bg + loc + key

html2 = response2.read()


1577

In [257]:
path = "/Users/sippiejason/datascience/webscraping/realestate/data/"

allAddrGeo = pd.read_csv(path + "addrGeo.csv", index_col=0)

allGeo = set(allAddrGeo['OBJECTID'])
print len(allGeo)

i = 0
allDemog = pd.DataFrame()

for geo in allGeo:

    t= allAddrGeo.ix[[geo]].iloc[[0],:]
    

    countyCode = str(t.ix[geo,"COUNTY"])
    stateCode = str(t.ix[geo,"STATE"])
    blockGroup = str(t.ix[geo,"BLKGRP"])
    tract = str(t.ix[geo,"TRACT"])

    fileNm = ""
    metric = ""
    prefix = "http://api.census.gov/data/2015/acs5?get=NAME,"
    key = '&key=49bdc2de261c9822e82e5e1c8c1b288ed173bd90'

    d = {'GeoID': geo}
    for k in demoDict.keys():
        fileNm = demoDict[k]["File"]
        for item in demoDict[k].keys():
            if item != "File":
                metric = demoDict[k][item]
                colLabel = k + item
                
                url = prefix + fileNm + metric + '&for=block+group:' + blockGroup + \
                        '&in=state:' + stateCode + '+county:0' + countyCode +\
                        '+tract:' + tract + key

                response = urllib2.urlopen(url)
                html = response.read()
                j = json.loads(html)

                d[colLabel] = int(j[1][1])
    i+=1
    print d
    allDemog = allDemog.append(d, ignore_index=True)
    
    if i == 3: break


#http://api.census.gov/data/2015/acs5?get=NAME,B01001_001E&for=block+group:2&in=state:37+county:087+tract:2021&key=49bdc2de261c9822e82e5e1c8c1b288ed173bd90              

540
{'HHIncomeMedIncome': 2093, 'HHwChildUnd18': 306, 'RetIncomeTotal': 1157, 'RaceTotal': 2760, 'EduAttainTotal': 2093, 'HHwChildTotal': 1157, 'HHwSeniorsTotal': 1157, 'RaceOther': 2700, 'HHwSeniorsOver65': 409, 'EduAttainBachelors': 220, 'MortStatNumWithMoreAnd': 0, 'MortStatNumWithMort': 484, 'EduAttainHS': 485, 'MortStatNumWithMoreOr': 111, 'MedValueMedValue': 136200, 'GeoID': 6979587, 'RetIncomeRetIncCnt': 299, 'MortStatTotal': 1017}
{'HHIncomeMedIncome': 1850, 'HHwChildUnd18': 224, 'RetIncomeTotal': 1011, 'RaceTotal': 2358, 'EduAttainTotal': 1850, 'HHwChildTotal': 1011, 'HHwSeniorsTotal': 1011, 'RaceOther': 2317, 'HHwSeniorsOver65': 694, 'EduAttainBachelors': 511, 'MortStatNumWithMoreAnd': 0, 'MortStatNumWithMort': 355, 'EduAttainHS': 287, 'MortStatNumWithMoreOr': 116, 'MedValueMedValue': 273500, 'GeoID': 6873094, 'RetIncomeRetIncCnt': 486, 'MortStatTotal': 577}
{'HHIncomeMedIncome': 1724, 'HHwChildUnd18': 372, 'RetIncomeTotal': 1109, 'RaceTotal': 2693, 'EduAttainTotal': 1724, 'H

In [219]:
response = urllib2.urlopen('http://api.census.gov/data/2015/acs5?get=NAME,B02001_001E&for=block+group:2&in=state:37+county:087+tract:2021&key=49bdc2de261c9822e82e5e1c8c1b288ed173bd90')

In [256]:
print allDemog


   EduAttainBachelors  EduAttainHS  EduAttainTotal      GeoID  \
0               220.0        485.0          2093.0  6979587.0   
1               511.0        287.0          1850.0  6873094.0   
2               609.0        219.0          1724.0  6876844.0   

   HHIncomeMedIncome  HHwChildTotal  HHwChildUnd18  HHwSeniorsOver65  \
0             2093.0         1157.0          306.0             409.0   
1             1850.0         1011.0          224.0             694.0   
2             1724.0         1109.0          372.0             213.0   

   HHwSeniorsTotal  MedValueMedValue  MortStatNumWithMoreAnd  \
0           1157.0          136200.0                     0.0   
1           1011.0          273500.0                     0.0   
2           1109.0          199600.0                     0.0   

   MortStatNumWithMoreOr  MortStatNumWithMort  MortStatTotal  RaceOther  \
0                  111.0                484.0         1017.0     2700.0   
1                  116.0                355

In [225]:

#r[1][1]
# total 107
# 87 with mortgage
# 20 with no mortgage

Server: Apache-Coyote/1.1
Cache-Control: max-age=60, must-revalidate
Access-Control-Allow-Origin: *
Date: Sat, 10 Dec 2016 14:56:17 GMT
Connection: close



In [188]:
demoDict = {'Race': {'File': 'B02001',
                     'Total': '_001E',
                     'Other': '_002E'},
            'HHwChild': {'File': 'B11005',
                         'Total': '_001E',
                         'Und18': '_002E'},
            'HHwSeniors': {'File': 'B11007',
                           'Total': '_001E',
                           'Over65': '_002E'},
            'EduAttain': {'File': 'B15003',
                          'Total': '_001E',
                          'HS': '_017E',
                          'Bachelors': '_022E'},
            'HHIncome': {'File': 'B15003',
                         'MedIncome': '_001E'},
            'RetIncome': {'File': 'B19059',
                          'Total': '_001E',
                          'RetIncCnt': '_002E'},
            'MortStat': {'File': 'B25081',
                         'Total': '_001E',
                         'NumWithMort': '_002E',
                         'NumWithMoreOr': '_003E',
                         'NumWithMoreAnd': '_006E'},
            'MedValue': {'File': 'B25077',
                         'MedValue': '_001E'}
            }




    # prefix = "http://api.census.gov/data/2015/acs5?get=NAME,"
    # bg = "&for=block+group:" & blockGroup
    # loc = "&in=state:" + stateCode + "+county:" + countyCode + "+tract:" + tractCode
    # key = "&key=49bdc2de261c9822e82e5e1c8c1b288ed173bd90"
    #
    # url = prefix + metric + bg + loc + key
    #
    # html2 = response2.read()


['Race',
 'MortStat',
 'MedValue',
 'EduAttain',
 'HHwChild',
 'HHIncome',
 'HHwSeniors',
 'RetIncome']

In [258]:
d = pd.DataFrame({"house":['aa','aa','bb','bb'],'year':[2001, 2000, 2001,2000], 'val':[200, 100, None, 100]} )
d = d.sort_values(["house",'year'])

In [263]:
gb = d.groupby('house')
print d
gb.ffill().diff()

  house    val  year
1    aa  100.0  2000
0    aa  200.0  2001
3    bb  100.0  2000
2    bb    NaN  2001


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [246]:
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1, 2, 3, 10, 20, 30], lst)


In [252]:
grouped = s.groupby(level=0)
print s
grouped.first()

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64


1    1
2    2
3    3
dtype: int64

In [82]:
# full constructor
df = pd.DataFrame(
    colums = [
        'ObsYear',
        'SaleFlg',
        'SalePrice',
        'ProxyValue',
        'AdjValue',
        'PctOfPrevSale',
        'Tax',
        'MortRate',
        'MortRateChgYoY',
        'YrsSinceRemodel',
        'YrsSinceSold',
        'YrdOld',
        'LocalUnempRate',
        'LocalUnempRateChgYoY',
        'PctOld',
        'PctOldChg',
        'PctYoung',
        'PctYoungChg',
        'Payroll',
        'PayrollPctChg',
        'LargestIndustPctChg',
        'LargestIndustPctChg5Y',
        'HouseID',
        'BuildYear',
        'CoolingType',
        'HeatingType',
        'LotSize',
        'RemodelYear',
        'SingleMulti',
        'NumBaths',
        'NumBeds',
        'Sqft'
        'ZipCode',
        'County'])

#Partial constructor
df = pd.DataFrame(
    colums = [
        'ObsYear',
        'YrsSinceSold',
        'SaleFlg',
        'SalePrice',
        'YrsSinceRemodel',
        'YrdOld',
        'HouseID',
        'BuildYear',
        'CoolingType',
        'HeatingType',
        'LotSize',
        'RemodelYear',
        'SingleMulti',
        'NumBaths',
        'NumBeds',
        'Sqft'])


TypeError: __init__() got an unexpected keyword argument 'colums'

In [299]:
inputPath = '/users/sippiejason/datascience/webscraping/realestate/data/'

houseObs = pd.read_csv(path + 'allHouseObservations.csv',index_col=False)
addrs = houseObs.loc[:,['AddrQuery', 'ZipCode','CityName','CountyName','StreetAddr', ]].drop_duplicates()
allGeos = pd.read_csv(path + "addrGeo.csv")


In [337]:
ignoreListFile = '/users/sippiejason/datascience/webscraping/realestate/data/AddrAlreadyLoaded.csv'
ignoreList = pd.read_csv(ignoreListFile, index_col=0)

False

In [334]:
neededGeos

,OBJECTID,COUNTY,STATE,BLKGRP,TRACT
1582,6981373,87,37,1,920800
1586,6979436,87,37,1,920900
1588,6978321,87,37,2,921000
1589,6979776,87,37,2,921100
1600,6979766,87,37,1,921301
1602,6978364,87,37,5,921200
1604,6981213,87,37,1,920800
1606,6980064,87,37,1,920800
1614,6980198,87,37,1,920800
1617,6981601,87,37,2,921000


In [314]:
neededGeos

,Unnamed: 0,AREALAND,AREAWATER,AddrQuery,BASENAME,BLKGRP,BLOCK,CENTLAT,CENTLON,COUNTY,...,MTFCC,NAME,OBJECTID,OID,POP100,STATE,SUFFIX,TRACT,UR,Unnamed: 0.1
1582,6981373,104383,0,44 Wildflower Ln Waynesville NC 28786,1135,1,1135,35.449578,-83.066417,87,...,G5040,Block 1135,6981373,2.104040e+14,15,37,NaN,920800,NaN,NaN
1586,6979436,11995,0,452 Chestnut Park Dr Waynesville NC 28786,1005,1,1005,35.497273,-82.997997,87,...,G5040,Block 1005,6979436,2.104040e+14,20,37,NaN,920900,NaN,NaN
1588,6978321,11563,0,464 Boundary St Waynesville NC 28786,2077,2,2077,35.494291,-82.982530,87,...,G5040,Block 2077,6978321,2.104040e+14,12,37,NaN,921000,NaN,NaN
1589,6979776,112377,0,2004 Pigeon Rd Waynesville NC 28786,2001,2,2001,35.473792,-82.960036,87,...,G5040,Block 2001,6979776,2.104040e+14,19,37,NaN,921100,NaN,NaN
1600,6979766,43060,0,4364 Pigeon Rd Waynesville NC 28786,1063,1,1063,35.471812,-82.918049,87,...,G5040,Block 1063,6979766,2.104040e+14,15,37,NaN,921301,NaN,NaN
1602,6978364,21961,0,293 Brook St Waynesville NC 28786,5033,5,5033,35.474016,-82.998932,87,...,G5040,Block 5033,6978364,2.104040e+14,28,37,NaN,921200,NaN,NaN
1604,6981213,10188,0,91 Wayneview Dr Waynesville NC 28786,1054,1,1054,35.490586,-83.029854,87,...,G5040,Block 1054,6981213,2.104040e+14,4,37,NaN,920800,NaN,NaN
1606,6980064,109870,0,930 Scenic Cir Waynesville NC 28786,1126,1,1126,35.486965,-83.027102,87,...,G5040,Block 1126,6980064,2.104040e+14,9,37,NaN,920800,NaN,NaN
1614,6980198,171016,0,695 Woody Ln Waynesville NC 28786,1062,1,1062,35.482583,-83.033613,87,...,G5040,Block 1062,6980198,2.104040e+14,20,37,NaN,920800,NaN,NaN
1617,6981601,26740,0,344 Boundary St Waynesville NC 28786,2078,2,2078,35.494328,-82.984285,87,...,G5040,Block 2078,6981601,2.104040e+14,17,37,NaN,921000,NaN,NaN


In [312]:
j['result']['addressMatches'][0]['geographies']['Census Blocks'][0]


{u'AREALAND': 343976,
 u'AREAWATER': 0,
 'AddrQuery': '149 Country Meadows Ln Waynesville NC 28786',
 u'BASENAME': u'2033',
 u'BLKGRP': u'2',
 u'BLOCK': u'2033',
 u'CENTLAT': u'+35.5146182',
 u'CENTLON': u'-082.8704302',
 u'COUNTY': u'087',
 u'FUNCSTAT': u'S',
 u'GEOID': u'370879204002033',
 u'HU100': 39,
 u'INTPTLAT': u'+35.5146182',
 u'INTPTLON': u'-082.8704302',
 u'LSADC': u'BK',
 u'LWBLKTYP': u'L',
 u'MTFCC': u'G5040',
 u'NAME': u'Block 2033',
 u'OBJECTID': 6978253,
 u'OID': 210403969570533,
 u'POP100': 89,
 u'STATE': u'37',
 u'SUFFIX': u'',
 u'TRACT': u'920400',
 u'UR': u''}